MIT License

Copyright (c) 2024 Mohammed Yasin

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

> **WARNING**
>
> Don't use "Run All". Run the cells one after another manually.

In [1]:
!git clone https://github.com/ultralytics/ultralytics

Cloning into 'ultralytics'...
remote: Enumerating objects: 42328, done.
remote: Counting objects: 100% (1895/1895), done.
remote: Compressing objects: 100% (998/998), done.
remote: Total 42328 (delta 1196), reused 1433 (delta 888), pack-reused 40433 (from 1)
Receiving objects: 100% (42328/42328), 32.28 MiB | 14.66 MiB/s, done.
Resolving deltas: 100% (31160/31160), done.


In [2]:
%cd ultralytics

/Users/timothylee/Desktop/Web Development/htx-hackx/yolo-live/ultralytics


/Users/timothylee/Desktop/Web Development/htx-hackx/yolo-live/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!git reset --hard abd391b63338b79cebee808b6679498dcc94956f

HEAD is now at abd391b63 `ultralytics 8.2.55` adaptive `tflite_support` logic (#14385)


In [5]:
# Download patch
!wget https://gist.githubusercontent.com/Y-T-G/8f4fc0b78a0a559a06fe84ae4f359e6e/raw/05a9086de3825e15ebef3830229273982c3ad9ec/add_head.patch

--2024-09-25 21:47:20--  https://gist.githubusercontent.com/Y-T-G/8f4fc0b78a0a559a06fe84ae4f359e6e/raw/05a9086de3825e15ebef3830229273982c3ad9ec/add_head.patch
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11258 (11K) [text/plain]
Saving to: ‘add_head.patch’

add_head.patch      100%[===================>]  10.99K  --.-KB/s    in 0s      

2024-09-25 21:47:22 (36.4 MB/s) - ‘add_head.patch’ saved [11258/11258]



In [6]:
!git apply add_head.patch

In [7]:
!pip install .

Processing /Users/timothylee/Desktop/Web Development/htx-hackx/yolo-live/ultralytics
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.2.55-py3-none-any.whl size=802098 sha256=50d9a240940e613ac190612c15ff3a52f2394e64a2989fbb3841e53068e7c517
  Stored in directory: /private/var/folders/6z/zxlp3m_x1vq0rt0vhl6k9wg40000gn/T/pip-ephem-wheel-cache-k__qy6eh/wheels/20/65/17/52ea74a594afc0e43464a2da7bad60b02904e059a138e5e54a
Successfully built ultralytics

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [8]:
%cd ..

/Users/timothylee/Desktop/Web Development/htx-hackx/yolo-live


In [17]:
from ultralytics import YOLO
import torch

In [19]:
import cv2
import time
 
# load YOLOv8n
model = YOLO('merged.pt')
cap = cv2.VideoCapture(1)
 
if not cap.isOpened():
    print("error")
    exit()
 
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
prev_time = 0
fps = 0
while True:
 
    ret, frame = cap.read()
    
    if not ret:
        print("error")
        break
 
    results = model(frame, device='cpu')
 
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        confidences = result.boxes.conf.cpu().numpy()
        class_ids = result.boxes.cls.cpu().numpy().astype(int)
        for i in range(len(boxes)):
            box = boxes[i]
            x1, y1, x2, y2 = map(int, box[:4])
            confidence = confidences[i]
            class_id = class_ids[i]
            label = result.names[class_id]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{label} {confidence:.2f}', (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
 
    curr_time = time.time()
    fps = 1 / (curr_time - prev_time)
    prev_time = curr_time
 
    cv2.putText(frame, f'FPS: {fps:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('YOLOv8n Real-time', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
cap.release()
cv2.destroyAllWindows()

c:\Users\ASUS\anaconda3\envs\hacx\Lib\site-packages\ultralytics\nn\tasks.py:785: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")



0: 480x640 1 0, 267.6ms
Speed: 16.3ms preprocess, 267.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 0s, 168.6ms
Speed: 5.1ms preprocess, 168.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 0, 1 80, 142.1ms
Speed: 1.0ms preprocess, 142.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 0, 136.5ms
Speed: 2.2ms preprocess, 136.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 0, 146.2ms
Speed: 1.8ms preprocess, 146.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 0s, 173.2ms
Speed: 2.0ms preprocess, 173.2ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 0s, 1 80, 159.9ms
Speed: 3.0ms preprocess, 159.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 0, 1 80, 160.9ms
Speed: 1.0ms preprocess, 160.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480,

KeyboardInterrupt: 